In [14]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
# Класс для полносвязного слоя нейронной сети
class DenseLayer:
    def __init__(self, input_size, output_size, activation=None):
        # Инициализация весов слоя с небольшими случайными значениями (равномерное распределение)
        self.weights = np.random.randn(input_size, output_size) * 0.01
        # Инициализация смещений слоя, все смещения устанавливаем в 0
        self.biases = np.zeros((1, output_size))
        # Присваиваем функцию активации
        self.activation = activation
        # Инициализация переменных для оптимизации с использованием моментов (momentum)
        self.velocity_w = np.zeros_like(self.weights)
        self.velocity_b = np.zeros_like(self.biases)

        # Для алгоритма оптимизации Adam
        self.m_w = np.zeros_like(self.weights)  
        self.v_w = np.zeros_like(self.weights)  
        self.m_b = np.zeros_like(self.biases)   
        self.v_b = np.zeros_like(self.biases)   
        self.t = 0  

    def forward(self, inputs):
        """
        Прямой проход через слой нейросети
        """
        self.inputs = inputs
        self.z = np.dot(inputs, self.weights) + self.biases  
        self.a = self.activation.forward(self.z) if self.activation else self.z  
        return self.a

    def backward(self, grad_output, learning_rate, optimizer, momentum=0.9, clip_value=1.0, beta1=0.9, beta2=0.999, epsilon=1e-8):
        """
        Обратный проход для вычисления градиентов и обновления параметров
        """
        # Если есть функция активации, вычисляем градиенты для неё
        grad_activation = self.activation.backward(grad_output, self.z) if self.activation else grad_output
        
        # Вычисляем градиенты для весов и смещений
        grad_weights = np.dot(self.inputs.T, grad_activation)
        grad_biases = np.sum(grad_activation, axis=0, keepdims=True)
        
        # Градиенты на входе слоя (нужны для предыдущих слоёв)
        grad_inputs = np.dot(grad_activation, self.weights.T)
        
        # Если используется обрезание градиентов, применяем его
        if optimizer == 'gradient_clipping':
            grad_weights = np.clip(grad_weights, -clip_value, clip_value)
            grad_biases = np.clip(grad_biases, -clip_value, clip_value)
        
        # Обновление параметров в зависимости от выбранного оптимизатора
        if optimizer == 'sgd':  # Stochastic Gradient Descent (Градиентный спуск)
            self.weights -= learning_rate * grad_weights
            self.biases -= learning_rate * grad_biases
        elif optimizer == 'momentum':  # Метод с моментумом
            self.velocity_w = momentum * self.velocity_w - learning_rate * grad_weights
            self.velocity_b = momentum * self.velocity_b - learning_rate * grad_biases
            self.weights += self.velocity_w
            self.biases += self.velocity_b
        elif optimizer == 'adam':  # Adam optimizer
            self.t += 1
            # Обновление моментов для весов и смещений
            self.m_w = beta1 * self.m_w + (1 - beta1) * grad_weights
            self.v_w = beta2 * self.v_w + (1 - beta2) * (grad_weights ** 2)
            self.m_b = beta1 * self.m_b + (1 - beta1) * grad_biases
            self.v_b = beta2 * self.v_b + (1 - beta2) * (grad_biases ** 2)

            # Коррекция смещения моментов
            m_w_hat = self.m_w / (1 - beta1 ** self.t)
            v_w_hat = self.v_w / (1 - beta2 ** self.t)
            m_b_hat = self.m_b / (1 - beta1 ** self.t)
            v_b_hat = self.v_b / (1 - beta2 ** self.t)

            # Обновление весов и смещений с учётом моментов
            self.weights -= learning_rate * m_w_hat / (np.sqrt(v_w_hat) + epsilon)
            self.biases -= learning_rate * m_b_hat / (np.sqrt(v_b_hat) + epsilon)
        
        return grad_inputs  

# Класс для активации ReLU
class ReLU:
    def forward(self, x):
        """
        Прямой проход для ReLU активации.
        """
        return np.maximum(0, x)

    def backward(self, grad_output, x):
        """
        Обратный проход для ReLU активации
        """
        return grad_output * (x > 0)  

# Класс для активации Sigmoid
class Sigmoid:
    def forward(self, x):
        """
        Прямой проход для Sigmoid активации
        """
        return 1 / (1 + np.exp(-x))  

    def backward(self, grad_output, x):
        """
        Обратный проход для Sigmoid активации
        """
        sigmoid_x = self.forward(x)  
        return grad_output * sigmoid_x * (1 - sigmoid_x)  

# Класс для активации Softmax
class Softmax:
    def forward(self, x):
        """
        Прямой проход для Softmax активации
        """
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)  

    def backward(self, grad_output, x):
        """
        Обратный проход для Softmax активации
        """
        return grad_output  


In [3]:
# Класс для нейронной сети
class NeuralNetwork:
    def __init__(self):
        """
        Инициализация нейронной сети
        """
        self.layers = []  
    
    def add_layer(self, layer):
        """
        Добавление слоя в нейронную сеть
        """
        self.layers.append(layer)  
    
    def forward(self, x):
        """
        Прямой проход через нейронную сеть
        """
        for layer in self.layers:
            x = layer.forward(x)  
        return x  
    
    def backward(self, loss_grad, learning_rate, optimizer='sgd', momentum=0.9, clip_value=1.0):
        """
        Обратный проход через нейронную сеть (обратное распространение ошибки)
        """
        for layer in reversed(self.layers):
            loss_grad = layer.backward(loss_grad, learning_rate, optimizer, momentum, clip_value)
    
    def train(self, X, y, loss_fn, loss_fn_deriv, epochs=100, learning_rate=0.01, optimizer='sgd', momentum=0.9, batch_size=None, clip_value=1.0):
        """
        Тренировка нейронной сети
        """
        for epoch in range(epochs):
            X, y = shuffle_data(X, y)
            
            if batch_size:
                batches = create_minibatches(X, y, batch_size)
            else:
                batches = [(X, y)]
            
            # Для каждого мини-батча:
            for X_batch, y_batch in batches:
                output = self.forward(X_batch)
                
                loss = loss_fn(y_batch, output)
                
                loss_grad = loss_fn_deriv(y_batch, output)
                
                self.backward(loss_grad, learning_rate, optimizer, momentum, clip_value)
            
            # Каждые 10 эпох выводим информацию о текущем значении ошибки
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")


In [4]:
# Функция для перемешивания данных
def shuffle_data(X, y):
    """
    Перемешивает данные X и соответствующие метки y случайным образом
    """
    indices = np.arange(X.shape[0])  
    np.random.shuffle(indices)  
    return X[indices], y[indices]  

# Функция для создания мини-батчей
def create_minibatches(X, y, batch_size):
    """
    Разбивает данные X и метки y на мини-батчи заданного размера
    """
    batches = []  
    for i in range(0, X.shape[0], batch_size):
        X_batch = X[i:i+batch_size]  
        y_batch = y[i:i+batch_size]  
        batches.append((X_batch, y_batch))  
    return batches  

# Функция для нормализации данных
def normalize_data(X):
    """
    Нормализует данные X по каждому признаку (по колонкам), чтобы они имели нулевое среднее и стандартное отклонение 1
    """
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)  


In [5]:
# Функция для вычисления среднеквадратичной ошибки (MSE)
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)  

# Функция для вычисления производной от MSE (градиент MSE)
def mse_loss_derivative(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_true.size  

# Функция для вычисления функции потерь кросс-энтропии
def cross_entropy_loss(y_true, y_pred):
    m = y_true.shape[0]  
    return -np.sum(y_true * np.log(y_pred + 1e-9)) / m  

# Функция для вычисления производной от функции потерь кросс-энтропии
def cross_entropy_loss_derivative(y_true, y_pred):
    return (y_pred - y_true) / y_true.shape[0]  


In [6]:
# Загрузка датасета Iris
data = load_iris()
X, y = data.data, data.target

# One-hot encoding для меток классов
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализация данных
X_train = normalize_data(X_train)
X_test = normalize_data(X_test)


In [7]:
# Создание и настройка нейросети
model = NeuralNetwork()
model.add_layer(DenseLayer(4, 10, activation=ReLU()))
model.add_layer(DenseLayer(10, 3, activation=Softmax()))

# Обучение модели
model.train(X_train, y_train, loss_fn=cross_entropy_loss, loss_fn_deriv=cross_entropy_loss_derivative,
            epochs=170, learning_rate=0.01, optimizer='momentum', batch_size=16)

# Проверка на тестовых данных
predictions = model.forward(X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)
accuracy = np.mean(predicted_classes == true_classes)
print(f"Test Accuracy(точность) на Iris датасете: {accuracy * 100:.2f}%")


Epoch 0, Loss: 1.10246263943977
Epoch 10, Loss: 0.9828027515517984
Epoch 20, Loss: 0.36336118037499543
Epoch 30, Loss: 0.28593004047673337
Epoch 40, Loss: 0.199333415409687
Epoch 50, Loss: 0.2531552356335597
Epoch 60, Loss: 0.10973883640342524
Epoch 70, Loss: 0.0570426667362058
Epoch 80, Loss: 0.03186251295449231
Epoch 90, Loss: 0.03241482967989205
Epoch 100, Loss: 0.12564495122381356
Epoch 110, Loss: 0.02648417588628122
Epoch 120, Loss: 0.0030304053244566477
Epoch 130, Loss: 0.12140965350124083
Epoch 140, Loss: 0.02311621302578687
Epoch 150, Loss: 0.1973675374711001
Epoch 160, Loss: 0.21033673413244972
Test Accuracy(точность) на Iris датасете: 96.67%


In [17]:
# Загрузка датасета MNIST с OpenML
mnist = datasets.fetch_openml('mnist_784', version=1)

# Данные и метки
X, y = mnist["data"].values, mnist["target"].astype(int)  # Преобразуем DataFrame в массив NumPy и метки в целые числа

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Преобразование данных
X_train = X_train / 255.0  # Нормализация изображений (в диапазоне [0, 1])
X_test = X_test / 255.0

# One-hot encoding для меток классов
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train.values.reshape(-1, 1))  # Преобразуем y_train в массив NumPy
y_test = encoder.transform(y_test.values.reshape(-1, 1))  # Преобразуем y_test в массив NumPy


In [18]:
# Создание и настройка нейросети
model = NeuralNetwork()
model.add_layer(DenseLayer(784, 256, activation=ReLU()))
model.add_layer(DenseLayer(256, 128, activation=ReLU()))
model.add_layer(DenseLayer(128, 10, activation=Softmax()))

# Обучение модели
model.train(X_train, y_train, 
            loss_fn=cross_entropy_loss, 
            loss_fn_deriv=cross_entropy_loss_derivative,
            epochs=50, 
            learning_rate=0.01, 
            optimizer='momentum', 
            momentum=0.9, 
            batch_size=64)

# Проверка на тестовых данных
predictions = model.forward(X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)
accuracy = np.mean(predicted_classes == true_classes)
print(f"Test Accuracy(точность) на MNIST датасете: {accuracy * 100:.2f}%")


Epoch 0, Loss: 0.33104928574330184
Epoch 10, Loss: 0.04653202634551339
Epoch 20, Loss: 0.007269233832305614
Epoch 30, Loss: 0.002240629861564179
Epoch 40, Loss: 0.0005680764779192366
Test Accuracy(точность) на MNIST датасете: 97.88%
